In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import cloudpickle as cp
sys.path.append("../py/")
from controller import *
from mlp import *
from data import *
import multiprocess as mp
import seaborn as sb
%matplotlib
#%config InlineBackend.figure_format = 'svg'

Using matplotlib backend: Qt5Agg


# Data

In [2]:
# state control data
rdata = np.load("../npy/moc_data_cart.npy")

# Neural Networks

In [3]:
# hidden shape
hshape = [50, 50, 50]

# nets
net = throttle(hshape, 7, drop=0)

NameError: name 'throttle' is not defined

In [ ]:
print(net(d.i), d.o[:, 0])

In [ ]:
net.train(d.i, d.o[:, 0], epo=1000, lr=1e-4, batches=1)

In [ ]:
for p in net.parameters():
    print(p)

In [ ]:
net.plot()

In [ ]:
y = d.o.numpy()
yp = net(d.i).data.numpy()

In [ ]:
fig, axs = plt.subplots(1, 4)

n = 1000
for ax, i in zip(axs, range(4)):
    ax.plot(y[:n, i], ".")
    ax.plot(yp[:n, i], "x")
    
plt.show()

## Parallel Training

In [ ]:
p = mp.Pool(3)

nets = p.map(
    lambda i: nets[i].train(
        d.i, d.o[:, i], epo=10, batches=10, ptst=0.1, lr=1e-2
    ),
    range(3)
)

In [ ]:
i = 0
nets[i].train(d.i, d.o[:, i], lr=1e-5)

In [ ]:
nets[1].plot()

In [ ]:
mpl.style.use('default')
fig, axs = plt.subplots(2, 3)
labels = list()
for net, lax, dax, i, unit in zip(nets, axs[0], axs[1], [7, 8, 9], ["Throttle [ND]", "Azimuth [rad]", "Polar [rad]"]):
    
    # loss
    lax = net.plot(lax)
    lax.legend(["Training Loss", "Testing Loss"])
    lax.set_title(net.name + " Network " + str(net.shape))
    lax.set_xlabel("Epoch")
    lax.set_ylabel("MSE Loss")
    
    # physical
    n = 400
    dax.plot(rdata[:n, i], ".")
    dax.plot(net(rdata[:n, :7]).data.numpy(), "x")
    dax.set_ylabel(unit)
    dax.legend(["Truth", "Predicted"])
    
fig.show()

In [ ]:
plt.figure()
plt.plot(rdata[:200, 7], "k.-")
plt.plot(nets[0](rdata[:200, :7]).data.numpy(), "x")
plt.show()

In [ ]:
nets[0].ltrn